In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Part 1 — Non-Linear Regression

In this section, we study the behavior of polynomial regression and ridge regularization when fitting a non-linear function.  
We will examine how different values of the regularization parameter λ affect model generalization, especially under a Leave-One-Out Cross-Validation (LOOCV) evaluation.

### Generating the Dataset

We generate **25 data points** using the function:

\[
y = sin(5 𝜋 xi) + 𝜀!
\]

where:
- \( x  ∈ (0, 1) \)
- \( 𝜀! ∈ (-0.3, 0.3) \)

We then build polynomial features up to degree 9, producing:

\[
X = [1, x, x^2, ..., x^9]
\]

Finally, we hold out **5 points** as a final **test set**, leaving **20 points** for cross-validation.

In [ ]:
np.random.seed(12)

n = 25
degree = 9

x = np.random.rand(n)  
eps = np.random.uniform(-0.3, 0.3, n)

y = np.sin(5 * np.pi * x) + eps
X = np.column_stack([x**k for k in range(degree + 1)])

X_temp, X_test, y_temp, y_test, x_temp, x_test = train_test_split(
    X, y, x, test_size=5, random_state=2)

X_temp.shape, X_test.shape

### Leave-One-Out Cross-Validation (LOOCV)

Since our dataset is very small, using a traditional train/validation split would be unreliable.  
Instead, for each value of λ, we perform LOOCV on the 20 non-test points:

1. Leave out one point for validation  
2. Train on the remaining 19  
3. Compute the squared error on the held-out point  
4. Repeat for all 20 points  
5. Average the validation errors to obtain the LOOCV error for that λ  

This process provides a more stable and unbiased estimate of model performance.

In [ ]:
lambdas = [0, 0.01, 0.1, 1, 10]
cv_errors = {}

X_cv = X_temp
y_cv = y_temp
x_cv = x_temp

for lam in lambdas:
    errors = []

    for i in range(len(X_cv)):
        X_train = np.delete(X_cv, i, axis=0)
        y_train = np.delete(y_cv, i, axis=0)

        X_val = X_cv[i].reshape(1, -1)
        y_val = y_cv[i]

        model = Ridge(alpha=lam, fit_intercept=False)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        errors.append((y_val - y_pred[0])**2)

    cv_errors[lam] = np.mean(errors)

cv_errors

### Selecting the Best Regularization Parameter λ

We compute CV_MSE for each tested value of λ.  
The λ with the lowest LOOCV error is chosen as the **optimal regularization strength**.

In [ ]:
best_lambda = min(cv_errors, key=cv_errors.get)
best_lambda

### Training the Final Model

Using the best λ from LOOCV, we refit the model on all 20 cross-validation points.  
This gives the model the maximum amount of training data while still preserving an unbiased estimate of λ.

In [ ]:
final_model = Ridge(alpha=best_lambda, fit_intercept=False)
final_model.fit(X_cv, y_cv)

### Evaluating on the Test Set

We now evaluate the chosen model on the 5 completely unseen test points.  
This provides the **true generalization error** of our final model.

In [ ]:
y_pred_test = final_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred_test)
test_mse

### Plotting the Polynomial Fits for Different λ

Here we visualize how the ridge regression model behaves for different λ values.  
- Small λ → flexible, wiggly curves (risk of overfitting)  
- Large λ → smoother, more biased curves  

The plot overlays:
- The 20 LOOCV points (blue)
- The 5 test points (red)
- The fitted curves for each λ

This allows us to visually compare model smoothness and stability.

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(x_cv, y_cv, color='blue', label='CV points (20)')
plt.scatter(x_test, y_test, color='red', label='Test points (5)')

grid = np.linspace(x.min(), x.max(), 600)
X_grid = np.column_stack([grid**k for k in range(degree + 1)])

for lam in lambdas:
    model = Ridge(alpha=lam, fit_intercept=False)
    model.fit(X_cv, y_cv)
    y_pred_grid = model.predict(X_grid)
    plt.plot(grid, y_pred_grid, label=f"λ = {lam}")

plt.title("Ridge Fits for Different λ Values (LOOCV on 20 points)")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

### Summary of Results

We print:

- The LOOCV MSE for each λ  
- The selected best λ  
- The final test MSE  

These values allow us to quantify:
- How each λ performed during cross-validation  
- How well the chosen model generalizes to new data  

In [ ]:
print("LOOCV Errors:")
for lam, err in cv_errors.items():
    print(f"lambda = {lam:<5}   CV MSE = {err:.6f}")

print("\nBest lambda:", best_lambda)
print("Final Test MSE:", test_mse)